In [1]:
# import the libraries

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np


2023-04-14 15:59:59.255736: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 15:59:59.903457: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/tensor/anaconda3/envs/tf/lib/:/lib
2023-04-14 15:59:59.903534: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/tensor/anaconda3/envs/tf/lib/:/lib
2023-04-14 15:59:59.903541: W tensorflow/compi

DATA LOADING

In [2]:
# Load the data
definitions = pd.read_csv('OPTED-Dictionary.csv')
simpDef1 = pd.read_excel('ChildFriendlyDefinitions.xlsx', sheet_name='Sheet1')
simpDef2 = pd.read_json('data.json')


DATA PREPROCESSING (Everything below this only needs to be run once. A CSV will be created with the full dataset)

In [3]:
import pandas as pd


def cleanDataframe(df):
    df_copy = df

    # may need to take the '-' out
    regex = "\[(.*?)\]|[0-9!@#$%^&*\/=+]|\((.*?)\)|\{(.*?)\}|\<(.*?)\>"

    df_copy = df_copy.replace(to_replace=regex, value="", regex=True)  # remove illegal chars
    
 
    df_copy = df_copy.replace(to_replace="-", value=" ", regex=True)
    df_copy = df_copy.replace(to_replace="_", value=" ", regex=True)

    df_copy = df_copy.replace(to_replace=",", value=" . ", regex=True)
    df_copy = df_copy.replace(to_replace="\"", value="", regex=True)
    df_copy = df_copy.replace(to_replace="\'", value="", regex=True)
    df_copy = df_copy.replace(to_replace="\?", value=" . ", regex=True)
    df_copy = df_copy.replace(to_replace="\.", value=" . ", regex=True)
    df_copy = df_copy.replace(to_replace="!", value=" . ", regex=True)
    df_copy = df_copy.replace(to_replace=";", value=" ; ", regex=True)
    
    
    for index, definition in df_copy.iterrows():
        #print(definition)
        word = definition['word']
        actualDefinition = definition['definition']

        #split and append to the dataset
        splitDefinition = actualDefinition.split(';') # should split on definition
        #print(splitDefinition)
        for index, splitDef in enumerate(splitDefinition):
            # Create a new dataset with the new defs that we split on the char ';'
             df_copy = pd.concat([df_copy, pd.DataFrame([[word, splitDef]], columns=['word', 'definition'])], ignore_index=True)

    df_copy.word = df_copy.word.str.lower()  # lower case everything
    df_copy.definition = df_copy.definition.str.lower()  # lower case everything

    df_copy = df_copy.sort_values('word', ascending=True)
    #df_copy = df_copy.drop_duplicates(subset='word', keep='first')

    return df_copy.reset_index().drop(['index'], axis=1)


'''
 df = pd.concat([df, pd.DataFrame([[w, defs, simpDefs]], columns=['word', 'definition', 'simplified_definition'])], ignore_index=True)

'''


"\n df = pd.concat([df, pd.DataFrame([[w, defs, simpDefs]], columns=['word', 'definition', 'simplified_definition'])], ignore_index=True)\n\n"


ACTUAL DEFINITIONS

In [4]:
definitions_filter = definitions.drop(['Count', 'POS'], axis=1)
definitions_filter['word'] = definitions_filter['Word']
definitions_filter['definition'] = definitions_filter['Definition']

definitions_filter = definitions_filter.drop(['Word', 'Definition'], axis=1)
definitions_filter = cleanDataframe(df=definitions_filter)

regexQuote = "^( \")|(\" )$"
definitions_filter = definitions_filter.replace(
    to_replace=regexQuote, value="", regex=True)
definitions_filter


,word,definition
0,. a . or an . pcp . it is presumabl...,specifically to go through with as a verse m...
1,. a . or an . pcp . it is presumabl...,to recite metrically .
2,. a . or an . pcp . it is presumabl...,to show in reading the metrical structure of
3,. a . or an . pcp . it is presumabl...,specifically to go through with as a verse m...
4,. a . or an . pcp . it is presumabl...,to examine with care
...,...,...
501878,NaN,used adjectively before a vowel in old s...
501879,NaN,as thou shalt have none assurance of thy lif...
501880,NaN,same as nones .
501881,NaN,a game played with such pieces


SIMPLIFIED DEFINITIONS

Part A

In [5]:
# Filter simpDef1
simpDef1_Filter = simpDef1.drop(['Exemplar'], axis=1)
simpDef1_Filter['word'] = simpDef1_Filter['Word']
simpDef1_Filter['definition'] = simpDef1_Filter['Child Friendly Definition']
simpDef1_Filter = simpDef1_Filter.drop(
    ['Word', 'Child Friendly Definition'], axis=1)
simpDef1_Filter = cleanDataframe(simpDef1_Filter)
simpDef1_Filter


,word,definition
0,accessible,when something is accessible it means anyone c...
1,accessible,when something is accessible it means anyone c...
2,accommodate,you accommodate when you change something that...
3,accommodate,you accommodate when you change something that...
4,accomplish,if you accomplish something . you succeed i...
...,...,...
333,vigilant,someone who is vigilant pays careful attention...
334,visible,when something is visible . you can see it .
335,visible,when something is visible . you can see it .
336,wish,when you wish for something . you think abo...


Part B

In [6]:
simpDef2_filter = simpDef2
simpDef2_Filter = cleanDataframe(simpDef2_filter)
simpDef2_Filter


,word,definition
0,able,a suffix that means capable or worthy of being...
1,able,a suffix that means capable or worthy of being...
2,al,a suffix that means of or having to do with .
3,al,a suffix that means the action of .
4,al,a suffix that means the action of .
...,...,...
32857,zoology,the science and study of animals .
32858,zoom,to move quickly while making a low humming sou...
32859,zoom,to move quickly while making a low humming sou...
32860,zucchini,a type of summer squash that is shaped like a ...


Getting Dataset ready for training

In [7]:
# Basically we are combining all the dataset together and putting it into a csv
# Doing this because i don't want to constantly run this script (takes up RAM that I dont have)

df = pd.DataFrame(columns=['word', 'definition', 'simplified_definition'])

for index, row in simpDef1_Filter.iterrows():
    defs = definitions_filter[row['word'] ==
                              definitions_filter.word]['definition']
    if defs.count() >= 1:
        w = row['word']
        simpDefs = row['definition']
        defs = defs.values[0]
        df = pd.concat([df, pd.DataFrame([[w, defs, simpDefs]], columns=[
                       'word', 'definition', 'simplified_definition'])], ignore_index=True)

for index, row in simpDef2_Filter.iterrows():
    defs = definitions_filter[row['word'] ==
                              definitions_filter.word]['definition']
    if defs.count() >= 1:
        w = row['word']
        simpDefs = row['definition']
        defs = defs.values[0]
        df = pd.concat([df, pd.DataFrame([[w, defs, simpDefs]], columns=[
                       'word', 'definition', 'simplified_definition'])], ignore_index=True)

df


,word,definition,simplified_definition
0,accessible,open to the influence of ; with to .,when something is accessible it means anyone c...
1,accessible,open to the influence of ; with to .,when something is accessible it means anyone c...
2,accommodate,as to accommodate a friend with a loan or wi...,you accommodate when you change something that...
3,accommodate,as to accommodate a friend with a loan or wi...,you accommodate when you change something that...
4,accomplish,to gain ; to obtain .,if you accomplish something . you succeed i...
...,...,...,...
25967,zone,circuit ; circumference .,an area that is divided from other areas becau...
25968,zoo,a combining form from gr . zwo n an animal as...,a prefix that means animal .
25969,zoo,a combining form from gr . zwo n an animal as...,a prefix that means animal .
25970,zoology,that part of biology which relates to the anim...,the science and study of animals .


In [8]:
df.to_csv("fullDataset.csv", index=False)
